In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')



tokenization

In [ ]:
import pandas as pd
import re
import html
import warnings
import os
import torch

from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

# -------------------------------
# Suppress warnings
# -------------------------------
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

# -------------------------------
# Paths
# -------------------------------
DATA_PATH = "/content/drive/MyDrive/NewsSumm Dataset.xlsx"
SAVE_DIR  = "/content/drive/MyDrive/tokenized_mistral_batches"

TRAIN_DIR = os.path.join(SAVE_DIR, "train")
TEST_DIR  = os.path.join(SAVE_DIR, "test")

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

# -------------------------------
# Load dataset
# -------------------------------
df = pd.read_excel(DATA_PATH)
df = df.dropna(subset=["human_summary"])
print("Rows after cleaning:", len(df))

# -------------------------------
# Resample
# -------------------------------
df = df.sample(n=15000, random_state=42)
print("Rows after resampling:", len(df))

# -------------------------------
# Clean text
# -------------------------------
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = BeautifulSoup(text, "html.parser").get_text(" ")
    text = html.unescape(text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", "", text)
    text = re.sub(r"\+?\d[\d\s\-]{8,}\d", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["human_summary"] = df["human_summary"].apply(clean_text)

# -------------------------------
# Train / Test split
# -------------------------------
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_texts = train_df["human_summary"].astype(str).tolist()
test_texts  = test_df["human_summary"].astype(str).tolist()

print("Train rows:", len(train_texts))
print("Test rows:", len(test_texts))

# -------------------------------
# Load Mistral tokenizer
# -------------------------------
MODEL_NAME = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True,
    trust_remote_code=True
)

# IMPORTANT for Mistral
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

# -------------------------------
# Tokenization settings (SAFE)
# -------------------------------
BATCH_SIZE = 4
MAX_LEN = 128

# -------------------------------
# Batch tokenization function
# -------------------------------
def tokenize_and_save(texts, out_dir, split_name):

    print(f"\nTokenizing {split_name} set...")

    batch_id = 0

    for i in range(0, len(texts), BATCH_SIZE):

        batch_texts = texts[i:i+BATCH_SIZE]

        encoded = tokenizer(
            batch_texts,
            truncation=True,
            padding="max_length",
            max_length=MAX_LEN,
            return_tensors="pt"
        )

        input_ids = encoded["input_ids"]
        attention_mask = encoded["attention_mask"]

        batch_data = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids.clone()
        }

        save_path = os.path.join(out_dir, f"batch_{batch_id}.pt")
        torch.save(batch_data, save_path)

        if batch_id % 50 == 0:
            print(f"{split_name}: saved batch {batch_id}")

        batch_id += 1

    print(f"{split_name} tokenization done. Total batches: {batch_id}")

# -------------------------------
# Run tokenization
# -------------------------------
tokenize_and_save(train_texts, TRAIN_DIR, "TRAIN")
tokenize_and_save(test_texts, TEST_DIR, "TEST")

print("\nMistral-7B batch-wise tokenization completed successfully")


In [ ]:
!pip install -q bitsandbytes peft accelerate


model training

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model

# --------------------
# Paths
# --------------------
TRAIN_DIR = "/content/drive/MyDrive/tokenized_mistral_batches/train/"
SAVE_DIR  = "/content/drive/MyDrive/mistral_qlora_finetuned"
MODEL_NAME = "mistralai/Mistral-7B-v0.1"

# --------------------
# Device
# --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --------------------
# Tokenizer
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# --------------------
# QLoRA: 4-bit config
# --------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# --------------------
# Load quantized model
# --------------------
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

# --------------------
# Add LoRA adapters
# --------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.train()

# --------------------
# Optimizer (only trains adapters)
# --------------------
optim = AdamW(model.parameters(), lr=2e-4)

# --------------------
# Load batch files
# --------------------
files = sorted([f for f in os.listdir(TRAIN_DIR) if f.endswith(".pt")])

total_loss = 0

# --------------------
# Training loop (same style)
# --------------------
for i, fname in enumerate(files, 1):

    batch = torch.load(os.path.join(TRAIN_DIR, fname))

    input_id  = batch["input_ids"].to(device)
    atten_id  = batch["attention_mask"].to(device)
    labels    = batch["labels"].to(device)

    optim.zero_grad()

    outputs = model(
        input_ids=input_id,
        attention_mask=atten_id,
        labels=labels
    )

    loss = outputs.loss
    loss.backward()

    optim.step()

    total_loss += loss.item()

    if i % 50 == 0:
        print(f"Loss at batch {i}: {total_loss / i:.4f}")

    del batch, input_id, atten_id, labels, outputs, loss
    torch.cuda.empty_cache()

# --------------------
# Save LoRA adapters
# --------------------
os.makedirs(SAVE_DIR, exist_ok=True)

model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print("QLoRA Mistral training completed and adapters saved")


In [ ]:
!pip install -q rouge-score


  Preparing metadata (setup.py) ... done


model evaluation

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from rouge_score import rouge_scorer

# --------------------
# Paths
# --------------------
TEST_DIR  = "/content/drive/MyDrive/tokenized_mistral_batches/test/"
ADAPTER_DIR = "/content/drive/MyDrive/mistral_qlora_finetuned"
BASE_MODEL = "mistralai/Mistral-7B-v0.1"

# --------------------
# Device
# --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --------------------
# Tokenizer
# --------------------
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# --------------------
# QLoRA 4-bit config
# --------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# --------------------
# Load base model (quantized)
# --------------------
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto"
)

# --------------------
# Load LoRA adapters
# --------------------
model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
model.eval()

# --------------------
# ROUGE scorer
# --------------------
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

r1 = r2 = rl = 0.0
r11 = r21 = rl1 = 0.0
count = 0

files = sorted([f for f in os.listdir(TEST_DIR) if f.endswith(".pt")])
print("Test batches:", len(files))

# --------------------
# Evaluation loop
# --------------------
for fname in files:

    batch = torch.load(os.path.join(TEST_DIR, fname))

    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"]

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50
        )

    gen_only = outputs[:, input_ids.shape[1]:]

    preds = tokenizer.batch_decode(gen_only, skip_special_tokens=True)
    refs  = tokenizer.batch_decode(labels, skip_special_tokens=True)

    for pred, ref in zip(preds, refs):
        scores = scorer.score(ref, pred)

        r1  += scores["rouge1"].fmeasure
        r2  += scores["rouge2"].fmeasure
        rl  += scores["rougeL"].fmeasure
        r11 += scores["rouge1"].precision
        r21 += scores["rouge2"].precision
        rl1 += scores["rougeL"].precision

        count += 1

# --------------------
# Final results
# --------------------
print("\nROUGE RESULTS (Mistral QLoRA)")
print("ROUGE-1 F1:", round(r1 / count, 4))
print("ROUGE-2 F1:", round(r2 / count, 4))
print("ROUGE-L F1:", round(rl / count, 4))
print("ROUGE-1 Precision:", round(r11 / count, 4))
print("ROUGE-2 Precision:", round(r21 / count, 4))
print("ROUGE-L Precision:", round(rl1 / count, 4))
